# Matching

In [1]:
import numpy as np
from importlib.resources import files, as_file
from at import (
    Lattice,
    LocalOpticsObservable,
    GlobalOpticsObservable,
    ObservableList,
    End,
)

Special import to use the new matching:

In [2]:
from at.future import RefptsVariable, VariableList, match

Import a sample lattice:

In [3]:
fname = "hmba.mat"
with as_file(files("machine_data") / fname) as path:
    ring = Lattice.load(path)

Split the sextupoles in halves to set constraints in their middle:

In [4]:
sf = ring.get_uint32_index("SF*")
sf1 = ring[sf[0]].divide([0.5, 0.5])
sf2 = ring[sf[1]].divide([0.5, 0.5])
ring.pop(sf[1])
ring.insert(sf[1], sf2[1])
ring.insert(sf[1], sf2[0])
ring.pop(sf[0])
ring.insert(sf[0], sf1[1])
ring.insert(sf[0], sf1[0])
ring.periodicity = 1

## Define the location of constraints

In [5]:
sf = ring.get_uint32_index("SF*")[1::2]
center = ring.get_uint32_index("CellCenter")

## Define the variables

We take the strengths of 7 quadrupole families

In [6]:
names = ["QF1*", "QD2*", "QD3*", "QF4*", "QD5*", "QF6*", "QF8*"]
bounds = [[0, 5], [-5, 0], [-5, 0], [0, 5], [-5, 0], [0, 5], [0, 5]]
variables = VariableList(
    RefptsVariable(nm, "PolynomB", index=1, bounds=bnd, name=nm, ring=ring)
    for nm, bnd in zip(names, bounds)
)

In [7]:
print(variables)


        Name      Initial          Final        Variation

        QF1*    2.539460e+00    2.539460e+00    0.000000e+00
        QD2*   -2.672025e+00   -2.672025e+00    0.000000e+00
        QD3*   -2.404266e+00   -2.404266e+00    0.000000e+00
        QF4*    2.429986e+00    2.429986e+00    0.000000e+00
        QD5*   -2.704501e+00   -2.704501e+00    0.000000e+00
        QF6*    4.464541e+00    4.464541e+00    0.000000e+00
        QF8*    4.425467e+00    4.425467e+00    0.000000e+00


## Define the constraints

Tunes:

In [8]:
obs = ObservableList()
# Tunes
obs.append(GlobalOpticsObservable("tune", target=[0.38, 0.85]))

Optics at the entrance of the lattice (1 reference point):

- $\alpha$ (both planes): the shape of the value is (1, 2). The shape of target (2,) is automatically broadcasted
  to (1, 2),
- $\eta'_x$: the shape of the value is (1,). The shape of target () is automatically broadcasted to (1,).

In [9]:
obs.append(LocalOpticsObservable(0, "alpha", target=[0.0, 0.0]))
obs.append(LocalOpticsObservable(0, "dispersion", plane="px", target=0.0))

Optics at the centre of the cell:

In [10]:
obs.append(LocalOpticsObservable(center, "beta", plane="y", target=4.69999257))
obs.append(LocalOpticsObservable(center, "alpha", target=[0.0, 0.0]))
obs.append(LocalOpticsObservable(center, "dispersion", plane="px", target=0.0))

Optics in the middle of sextupoles (2 reference points):

- $\beta_y$: the shape of the value is (2,). The target (shape (2,)) specifies the value at each refpoint,
- $\alpha_y$: the shape of the value is (2,). The target (shape (2,)) specifies the value at each refpoint,
- $\eta_x$: the shape of the value is (2,). The target (shape (2,)) specifies the value at each refpoint,

In [11]:
obs.append(LocalOpticsObservable(sf, "beta", plane="y", target=[5.4, 5.4]))
obs.append(
    LocalOpticsObservable(sf, "alpha", plane="y", target=[0.68000392, -0.67999686])
)
obs.append(
    LocalOpticsObservable(sf, "dispersion", plane="x", target=[0.08820002, 0.08819999])
)

Phase advance between the sextupoles (2 reference points):

- $\Delta\mu$ (both planes): the numpy {py:func}`~numpy.diff` function reduces the 1st dimension by 1,
  so the shape of value is (1, 2). The shape of target (2,) is broadcasted to (1, 2).

In [12]:
obs.append(
    LocalOpticsObservable(
        sf, "mu2pif", target=[0.49721338, 0.48228011], statfun=np.diff
    )
)

## Perform the matching:

In [13]:
newring = match(ring, variables, obs, copy=True)


17 constraints, 7 variables, using method trf

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.0793e-05                                    6.22e-03    
       1              3         5.4659e-06      5.33e-06       3.56e-03       2.33e-02    
       2              4         1.9162e-06      3.55e-06       7.17e-03       8.81e-02    
       3              5         9.4297e-08      1.82e-06       2.23e-03       4.15e-03    
       4              6         3.5981e-08      5.83e-08       7.08e-03       1.41e-03    
       5              7         2.8063e-09      3.32e-08       1.02e-02       3.01e-03    
       6              8         2.1677e-10      2.59e-09       1.20e-03       4.16e-05    
       7              9         1.7285e-10      4.39e-11       4.10e-04       4.88e-06    
       8             10         1.7284e-10      6.16e-15       2.11e-06       3.41e-10    
`gtol` termination condition is satisfied.